# **MediMateAI App Notebook**

## Introduction

Welcome to the MediMateAI notebook! This notebook is designed to showcase the functionality of the MediMateAI app, a powerful tool that utilizes Vertex AI and Google Vision to extract text from prescriptions. The app then uses this information to create reminders for users, providing details about each prescribed medicine, its side effects, and potential conflicts with medications the user is already taking.

## Features

### 1. Prescription Text Extraction

The app employs Google Vision to extract text from prescriptions. This ensures accurate and reliable information retrieval from medical documents.

### 2. Reminder Creation

Based on the extracted text, the app generates reminders for each prescribed medicine. These reminders include information about dosage, timing, and other relevant details.

### 3. Side Effects Information

MediMateAI informs users about the potential side effects of each prescribed medicine. This empowers users with valuable knowledge about their medications.

### 4. Medication Conflict Notifications

The app actively checks for conflicts between newly prescribed medications and those already taken by the user. If a conflict is detected, the user is promptly notified to ensure their safety.

## Getting Started

To begin using the notebook, make sure you have the necessary dependencies installed. You may also need to authenticate with Google services if required. Follow the steps outlined in the code cells for a seamless experience.

## Instructions

1. Upload a clear image of your prescription.
2. Run the provided code cells to extract text using Google Vision.
3. Review the generated result, side effects information, and conflict notifications.

## Important Note

This notebook is for demonstration purposes and may require customization for pordiction use. Ensure that you have the required API keys, permissions, and access to Google services.

Let's get started on the journey to smarter medication management with MediMateAI!


In [1]:
! pip install -q google-cloud-aiplatform==1.36.3 litellm==1.11.1 trulens_eval==0.18.3 langchain==0.0.347
!pip install -q google-cloud-vision
!pip install -q --upgrade google-cloud-documentai
!pip install -q faiss-cpu tiktoken
!pip install huggingface -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.0/627.0 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 929.3/929.3 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.7/194.7 kB 17.6 MB

In [ ]:
from google.oauth2 import service_account
from IPython.display import JSON
# Imports main tools:
from trulens_eval import TruChain, Feedback, Tru, LiteLLM
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import json
from google.api_core.client_options import ClientOptions
from google.cloud import documentai
from langchain.chains import LLMChain
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate
from google.cloud import aiplatform
from langchain.embeddings import VertexAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from vertexai.language_models import TextEmbeddingModel
import base64
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
import functools
import time
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm
import math
from vertexai.language_models import TextEmbeddingModel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Markdown, display
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma




## Init Trulens

In [ ]:
tru = Tru()
tru.reset_database()


**Load GCP credentials**

In [ ]:
credentials = service_account.Credentials.from_service_account_file("/content/root-habitat-258913-8104543cb5cc.json")

In [ ]:
aiplatform.init(credentials=credentials, project=credentials.project_id)

**OCR Prescription**

In [ ]:
def process_prescription(
    project_id: str,
    location: str,
    processor_id: str,
    file_path: str,
    mime_type: str,
    field_mask = None,
    processor_version_id= None,
) -> str:
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts, credentials= credentials)

    if processor_version_id:
        name = client.processor_version_path(
            project_id, location, processor_id, processor_version_id
        )
    else:

        name = client.processor_path(project_id, location, processor_id)

    # Read the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Load binary data
    raw_document = documentai.RawDocument(content=image_content, mime_type=mime_type)
    process_options = documentai.ProcessOptions(
        individual_page_selector=documentai.ProcessOptions.IndividualPageSelector(
            pages=[1]
        )
    )

    request = documentai.ProcessRequest(
        name=name,
        raw_document=raw_document,
        field_mask=field_mask,
        process_options=process_options,
    )

    result = client.process_document(request=request)
    document = result.document


    return document.text


In [ ]:
prescription = process_prescription(
  project_id="682811873214",
  location="us",
  processor_id="5b9c8c8527df3350",
  file_path="/content/images.jpeg",
  mime_type = 'image/jpeg'
)
print(prescription)

دكتـور
معـاذ عطيتـو
طبيب وجراح العيون والمياه البيضاء والليزك
جامعـة الأزهـر - مستشفى الأقصر الدولي
الإسم : أشرف أدهـم سـيف
السـن 11
2
Moaz Ateto
التاريخ : ٢٤ / ۱۱ / ۲۰۲۳م
Lubricant Eye Drops E.D.
قطرة 3 مرات يوميا لمدة أسبوعين
RI
R/Analgesic Drug TAB.
قرص بعد الأكل مرتين يوميـا
لمدة أسبوع " ولا يكرر
RI Antibiotic Eye Ointment. E.O.
مرهــم مسـاء المـدة أسبوع )



**Generate JSON Response Containing Medicine Name and Recommended Dosage**

In this section, we will discuss how to use extracted text to make the JSON response that the MediMateAI app needs, which contains crucial information about prescribed medicines, including the medicine name and the recommended dosage.
JSON Response Structure

The JSON response is structured to provide clear and organized data for each prescribed medicine. It consists of two key components:

    Medicine Name: Clearly identifies the name of the prescribed medicine.
    Dosage Information: Specifies the recommended dosage and the frequency at which the medicine should be taken.



In [ ]:
medical_prescription_template = """\
For the following text, extract the following information:
- medicine name : list of medicine names that are mentioned in the text
- taken number : number of times a day the medicine should be taken for each medicine name
text: {text}
<< FORMATTING >>
Return a markdown code snippet with a Dict object formatted to look like:
```{{
    "medicine": "taken_num"}}
    ```
"""

In [ ]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=medical_prescription_template,
        input_variables=["text"],
    )
)


In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])


In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = VertexAI()

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)



In [ ]:
llm_response = chain(prescription)

display(llm_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: For the following text, extract the following information:
- medicine name : list of medicine names that are mentioned in the text
- taken number : number of times a day the medicine should be taken for each medicine name
text: دكتـور
معـاذ عطيتـو
طبيب وجراح العيون والمياه البيضاء والليزك
جامعـة الأزهـر - مستشفى الأقصر الدولي
الإسم : أشرف أدهـم سـيف
السـن 11
2
Moaz Ateto
التاريخ : ٢٤ / ۱۱ / ۲۰۲۳م
Lubricant Eye Drops E.D.
قطرة 3 مرات يوميا لمدة أسبوعين
RI
R/Analgesic Drug TAB.
قرص بعد الأكل مرتين يوميـا
لمدة أسبوع " ولا يكرر
RI Antibiotic Eye Ointment. E.O.
مرهــم مسـاء المـدة أسبوع )

<< FORMATTING >>
Return a markdown code snippet with a Dict object formatted to look like:
```{
    "medicine": "taken_num"}
    ```


> Finished chain.


{'text': ' ```python\n{\n    "Lubricant Eye Drops E.D.": "3 times a day for two weeks",\n    "Analgesic Drug TAB.": "once after meal twice a day for a week",\n    "Antibiotic Eye Ointment. E.O.": "once at night for a week"\n}\n```'}

In [ ]:
llm_response

{'text': ' ```python\n{\n    "Lubricant Eye Drops E.D.": "3 times a day for two weeks",\n    "Analgesic Drug TAB.": "once after meal twice a day for a week",\n    "Antibiotic Eye Ointment. E.O.": "once at night for a week"\n}\n```'}

In [ ]:
# prompt: load llm_response"{'text': ' ```json\n{\n  "medicine": [\n    {\n      "medicine_name": "Lubricant Eye Drops",\n      "taken_num": "3 times a day for two weeks"\n    },\n    {\n      "medicine_name": "Analgesic Drug TAB",\n      "taken_num": "once after meal, twice a day for a week"\n    },\n    {\n      "medicine_name": "Antibiotic Eye Ointment",\n      "taken_num": "once at night for a week"\n    }\n  ]\n}\n```'}" into json ""
uncleaned_data = llm_response["text"].split('\n')

In [ ]:
uncleaned_data[2:-2]

['    "Lubricant Eye Drops E.D.": "3 times a day for two weeks",',
 '    "Analgesic Drug TAB.": "once after meal twice a day for a week",',
 '    "Antibiotic Eye Ointment. E.O.": "once at night for a week"']

## Load Drag Side Effect Dataset

In [ ]:
def convert_text_to_dict(text):
    try:
        text_str = ' '.join(text)

        dictionary = eval('{' + text_str + '}')
        return dictionary
    except Exception as e:
        print(f"Error converting text to dictionary: {e}")
        return None

input_text = uncleaned_data[2:-2]

cleaned_data = convert_text_to_dict(input_text)

if cleaned_data:
    print(cleaned_data)


{'Lubricant Eye Drops E.D.': '3 times a day for two weeks', 'Analgesic Drug TAB.': 'once after meal twice a day for a week', 'Antibiotic Eye Ointment. E.O.': 'once at night for a week'}


In [ ]:
df = pd.read_csv('/content/drugs_side_effects_drugs_com.csv')

In [ ]:
df.head()

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,activity,rx_otc,pregnancy_category,csa,alcohol,related_drugs,medical_condition_description,rating,no_of_reviews,drug_link,medical_condition_url
0,doxycycline,Acne,"(hives, difficult breathing, swelling in your ...",doxycycline,"Miscellaneous antimalarials, Tetracyclines","Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alod...",87%,Rx,D,N,X,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,6.8,760.0,https://www.drugs.com/doxycycline.html,https://www.drugs.com/condition/acne.html
1,spironolactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...","Aldactone, CaroSpir",82%,Rx,C,N,X,amlodipine: https://www.drugs.com/amlodipine.h...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.2,449.0,https://www.drugs.com/spironolactone.html,https://www.drugs.com/condition/acne.html
2,minocycline,Acne,"skin rash, fever, swollen glands, flu-like sym...",minocycline,Tetracyclines,"Dynacin, Minocin, Minolira, Solodyn, Ximino, V...",48%,Rx,D,N,NaN,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,5.7,482.0,https://www.drugs.com/minocycline.html,https://www.drugs.com/condition/acne.html
3,Accutane,Acne,problems with your vision or hearing; muscle o...,isotretinoin (oral),"Miscellaneous antineoplastics, Miscellaneous u...",NaN,41%,Rx,X,N,X,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.9,623.0,https://www.drugs.com/accutane.html,https://www.drugs.com/condition/acne.html
4,clindamycin,Acne,hives ; difficult breathing; swelling of your ...,clindamycin topical,"Topical acne agents, Vaginal anti-infectives","Cleocin T, Clindacin ETZ, Clindacin P, Clindag...",39%,Rx,B,N,NaN,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.4,146.0,https://www.drugs.com/mtm/clindamycin-topical....,https://www.drugs.com/condition/acne.html


In [ ]:
med_db = df[['drug_name', 'medical_condition', 'side_effects', 'generic_name']]


In [ ]:
med_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2931 entries, 0 to 2930
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   drug_name          2931 non-null   object
 1   medical_condition  2931 non-null   object
 2   side_effects       2807 non-null   object
 3   generic_name       2888 non-null   object
dtypes: object(4)
memory usage: 91.7+ KB


In [ ]:
med_db.dropna(inplace=True)

<ipython-input-297-ec38586f0efb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  med_db.dropna(inplace=True)


In [ ]:
med_db.head(2)

,drug_name,medical_condition,side_effects,generic_name
0,doxycycline,Acne,"(hives, difficult breathing, swelling in your ...",doxycycline
1,spironolactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone


In [ ]:
small_med_db = med_db.head(400)

In [ ]:
small_med_db.to_csv("small_drugs_side_effect.csv")

In [ ]:
medical = small_med_db.drug_name_side_effects_generic_name.tolist()

## Embedding data

In [ ]:
def generate_batches(sentences, batch_size = 5):
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]

def encode_texts_to_embeddings(sentences):
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]

def encode_text_to_embedding_batched(sentences, api_calls_per_second = 0.33, batch_size = 5):
    # Generates batches and calls embedding API

    embeddings_list = []

    # Prepare the batches using a generator
    batches = generate_batches(sentences, batch_size)

    seconds_per_job = 1 / api_calls_per_second

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(
            batches, total = math.ceil(len(sentences) / batch_size), position=0
        ):
            futures.append(
                executor.submit(functools.partial(encode_texts_to_embeddings), batch)
            )
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())

    is_successful = [
        embedding is not None for sentence, embedding in zip(sentences, embeddings_list)
    ]
    embeddings_list_successful = np.squeeze(
        np.stack([embedding for embedding in embeddings_list if embedding is not None])
    )
    return embeddings_list_successful

In [ ]:
embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@001")

In [ ]:
medical = small_med_db.drug_name_side_effects_generic_name.tolist()

In [ ]:
print(medical[0])

doxycycline, (hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low white blood cell counts - fever, chills, swollen glands, body aches, weakness, pale skin, easy bruising or bleeding; severe headaches, ringing in your ears, dizziness, nausea

In [ ]:
medical_embeddings = encode_text_to_embedding_batched(
            sentences = medical[:20],
            api_calls_per_second = 20/60,
            batch_size = 200)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import pickle
with open('medical_embeddings.pickle', 'wb') as handle:
    pickle.dump(medical_embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
with open('medical_embeddings.pickle', 'rb') as file:

    drugs_embeddings = pickle.load(file)

    print(drugs_embeddings)

[[-0.00672768 -0.02730319  0.00240885 ... -0.02212393 -0.01868371
  -0.02210676]
 [-0.00922261 -0.02114555 -0.01299223 ... -0.03240846  0.00854058
  -0.03885769]
 [ 0.00908843 -0.02520366  0.01095423 ... -0.048889   -0.00353505
  -0.01247655]
 ...
 [ 0.01638014 -0.02556225 -0.00073034 ... -0.00179679 -0.00135262
  -0.01433476]
 [-0.01633849 -0.00181386 -0.00151316 ... -0.02593947  0.01989759
   0.0079709 ]
 [ 0.01365166 -0.00146412 -0.0140797  ... -0.0480246  -0.00359442
  -0.03482459]]


In [ ]:
query = ["What is the side effect of doxycycline"]
query_embedding = embedding_model.get_embeddings(query)[0].values
cos_sim_array = cosine_similarity([query_embedding],
                                  drugs_embeddings)

In [ ]:
cos_sim_array.shape

(1, 20)

In [ ]:
index_doc_cosine = np.argmax(cos_sim_array)
small_med_db.drug_name_side_effects_generic_name[index_doc_cosine]

'doxycycline, (hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low white blood cell counts - fever, chills, swollen glands, body aches, weakness, pale skin, easy bruising or bleeding; severe headaches, ringing in your ears, dizziness, nause

'doxycycline, (hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low white blood cell counts - fever, chills, swollen glands, body aches, weakness, pale skin, easy bruising or bleeding; severe headaches, ringing in your ears, dizziness, nause

In [ ]:
context = "Question: " + query[0] +\
"\n Answer: " + small_med_db.drug_name_side_effects_generic_name[index_doc_cosine]

In [ ]:
prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """

In [ ]:
side_effect_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=prompt,
        input_variables=["context", "query"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

chat_prompt_template = ChatPromptTemplate.from_messages([side_effect_prompt])

llm = VertexAI()

chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)


llm_response = chain({"context":context, "query": query})

display(llm_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Here is the context: Question: What is the side effect of doxycycline
 Answer: doxycycline, (hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low white blood cell counts - fe

{'context': 'Question: What is the side effect of doxycycline\n Answer: doxycycline, (hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low white blood cell counts - fever, chills, swollen glands, body aches, weakness, pale skin, easy bruisin

In [ ]:

display(Markdown(llm_response["text"]))

 Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low white blood cell counts - fever, chills, swollen glands, body aches, weakness, pale skin, easy bruising or bleeding; severe headaches, ringing in your ears, dizziness, nausea, vision problems, pain behind your eyes; loss of appetite, upper stomach pain (that may spread to your back), tiredness, nausea or vomiting, fast heart rate, dark urine, jaundice (

In [ ]:
litellm = LiteLLM(model_engine="chat-bison")
relevance = Feedback(litellm.relevance_with_cot_reasons).on_input_output()
feed1=Feedback(litellm.criminality_with_cot_reasons).on_output()
feed2=Feedback(litellm.correctness_with_cot_reasons).on_output()
feed3=Feedback(litellm.sentiment_with_cot_reasons).on_output()
feed4=Feedback(litellm.insensitivity).on_output()
feed5=Feedback(litellm.coherence).on_output()

✅ In relevance_with_cot_reasons, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance_with_cot_reasons, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In criminality_with_cot_reasons, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In correctness_with_cot_reasons, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In sentiment_with_cot_reasons, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In insensitivity, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In coherence, input text will be set to __record__.main_output or `Select.RecordOutput` .


In [ ]:
tru_recorder = TruChain(chain,app_id='MediMate',feedbacks=[relevance,feed1,feed2,feed3,feed4, feed5])

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all
tru.run_dashboard() # open

Starting dashboard ...
npx: installed 22 in 4.427s

Go to this url and submit the ip given here. your url is: https://bright-monkeys-relax.loca.lt

  Submit this IP Address: 35.238.188.203



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
tru.stop_dashboard()

Dashboard closed.
Dashboard closed.


In [ ]:
with tru_recorder as recording:
  llm_response = chain({"context":context, "query": query})
display(llm_response)





> Entering new LLMChain chain...
Prompt after formatting:
Human: Here is the context: Question: What is the side effect of doxycycline
 Answer: doxycycline, (hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low white blood cell counts - fe

ERROR:trulens_eval.feedback.provider.endpoint.base:litellm request failed <class 'litellm.exceptions.APIError'>=VertexAIException - 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: chat-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas.. Retries remaining=2.



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.




> Finished chain.


ERROR:trulens_eval.feedback.provider.endpoint.base:litellm request failed <class 'litellm.exceptions.APIError'>=VertexAIException - 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: chat-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas.. Retries remaining=0.
ERROR:trulens_eval.feedback.provider.endpoint.base:litellm request failed <class 'RuntimeError'>=API litellm request failed 4 time(s).. Retries remaining=2.



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



{'context': 'Question: What is the side effect of doxycycline\n Answer: doxycycline, (hives, difficult breathing, swelling in your face or throat) or a severe skin reaction (fever, sore throat, burning in your eyes, skin pain, red or purple skin rash that spreads and causes blistering and peeling). Seek medical treatment if you have a serious drug reaction that can affect many parts of your body. Symptoms may include: skin rash, fever, swollen glands, flu-like symptoms, muscle aches, severe weakness, unusual bruising, or yellowing of your skin or eyes. This reaction may occur several weeks after you began using doxycycline. Doxycycline may cause serious side effects. Call your doctor at once if you have: severe stomach pain, diarrhea that is watery or bloody; throat irritation, trouble swallowing; chest pain, irregular heart rhythm, feeling short of breath; little or no urination; low white blood cell counts - fever, chills, swollen glands, body aches, weakness, pale skin, easy bruisin

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0]

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,...,sentiment_with_cot_reasons,relevance_with_cot_reasons_calls,insensitivity_calls,criminality_with_cot_reasons_calls,coherence_calls,correctness_with_cot_reasons_calls,sentiment_with_cot_reasons_calls,latency,total_tokens,total_cost
0,MediMate,"{""app_id"": ""MediMate"", ""tags"": ""-"", ""metadata""...",LLMChain(langchain.chains.llm),record_hash_e8556774a0cd597beed898e1bf6854c7,null,""" Doxycycline may cause serious side effects. ...",-,"{""record_id"": ""record_hash_e8556774a0cd597beed...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-08T15:38:24.006867"", ""...",...,0.0,"[{'args': {'prompt': None, 'response': ' Doxyc...",[{'args': {'text': ' Doxycycline may cause ser...,[{'args': {'text': ' Doxycycline may cause ser...,[{'args': {'text': ' Doxycycline may cause ser...,[{'args': {'text': ' Doxycycline may cause ser...,[{'args': {'text': ' Doxycycline may cause ser...,2,0,0.0
1,MediMate,"{""app_id"": ""MediMate"", ""tags"": ""-"", ""metadata""...",LLMChain(langchain.chains.llm),record_hash_7626dd9d3546644dc0c47b03f08fe882,null,""" Doxycycline may cause several side effects, ...",-,"{""record_id"": ""record_hash_7626dd9d3546644dc0c...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-08T15:38:58.325658"", ""...",...,NaN,[],[],[{'args': {'text': ' Doxycycline may cause sev...,[],NaN,[],2,0,0.0
2,MediMate,"{""app_id"": ""MediMate"", ""tags"": ""-"", ""metadata""...",LLMChain(langchain.chains.llm),record_hash_e4f0cb16b22ce0b46c5d48ddf534b770,null,""" Doxycycline may cause serious side effects. ...",-,"{""record_id"": ""record_hash_e4f0cb16b22ce0b46c5...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-08T15:39:28.038079"", ""...",...,0.0,"[{'args': {'prompt': None, 'response': ' Doxyc...",NaN,[{'args': {'text': ' Doxycycline may cause ser...,[{'args': {'text': ' Doxycycline may cause ser...,NaN,[{'args': {'text': ' Doxycycline may cause ser...,2,0,0.0
3,MediMate,"{""app_id"": ""MediMate"", ""tags"": ""-"", ""metadata""...",LLMChain(langchain.chains.llm),record_hash_381c7285ab1f04a2eb10d41e511ae56b,null,""" Doxycycline may cause serious side effects. ...",-,"{""record_id"": ""record_hash_381c7285ab1f04a2eb1...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-08T15:39:36.344881"", ""...",...,NaN,NaN,[{'args': {'text': ' Doxycycline may cause ser...,NaN,NaN,NaN,NaN,2,0,0.0


Explore in a Dashboard


In [ ]:
tru.run_dashboard()

Starting dashboard ...
npx: installed 22 in 5.663s

Go to this url and submit the ip given here. your url is: https://big-nights-walk.loca.lt

  Submit this IP Address: 34.16.134.141



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
tru.stop_dashboard() # stop if needed

Dashboard closed.Dashboard closed.



In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,relevance_with_cot_reasons,relevance_with_cot_reasons_calls,latency,total_tokens,total_cost
0,Chain1_ChatApplication,"{""app_id"": ""Chain1_ChatApplication"", ""tags"": ""...",LLMChain(langchain.chains.llm),record_hash_0d331e4479bf5a18a21ab49cd4bc61f7,"""\u062f\u0643\u062a\u0640\u0648\u0631\n\u0645\...",""" ```python\n{\n \""Lubricant Eye Drops E.D....",-,"{""record_id"": ""record_hash_0d331e4479bf5a18a21...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-07T22:09:46.968724"", ""...",2023-12-07T22:09:49.136758,1.0,[{'args': {'prompt': 'دكتـور معـاذ عطيتـو طبيب...,2,0,0.0
